In [ ]:
from IPython.display import display

import pandas as pd
from datetime import *
import os
import shutil
from sqlalchemy import create_engine
from ratio_sql import ratio_calculation



In [ ]:
db_name = 'test_stock02.db'
# os.remove(db_name) if os.path.exists(db_name) else None
dtb = f'sqlite:///{db_name}'
engine = create_engine(dtb)
ticker_list =['FPT' ,'VNM', 'HPG', 'MSN', 'GAS', 'GVR', 'MWG', 'PLX', 'VRE','VIC']
if os.path.exists(db_name):
    os.remove(db_name)
for ticker in ticker_list: 
    df_ratio = ratio_calculation(ticker)
    df_ratio = df_ratio.set_index('Chỉ số')
    df_ratio.loc['Ticker'] = ticker
    df_ratio = df_ratio.T
    df_ratio.index.name = 'year'
    df_ratio.to_sql('financial_ratio', engine, if_exists='append', index = True)

if os.path.exists('data_processed'):
    shutil.rmtree('data_processed')

if os.path.exists('data_raw'):
    shutil.rmtree('data_raw')


In [37]:

# bắt đầu lọc ra 5 cổ phiếu có P/E rẻ nhất (năm mới nhất)
query_cheap_pe = """
WITH LatestData AS (
    SELECT 
        Ticker,
        year,
        "P/E",
        ROW_NUMBER() OVER(PARTITION BY Ticker ORDER BY year DESC) as rn 
    FROM 
        financial_ratio
    WHERE 
        "P/E" IS NOT NULL
)
SELECT 
    Ticker,
    year,
    "P/E"
FROM 
    LatestData
WHERE 
    rn = 1 
    AND "P/E" > 0  -- Chỉ lấy P/E "rẻ", không lấy P/E âm (lỗ)
ORDER BY 
    "P/E" ASC  -- Sắp xếp P/E tăng dần
LIMIT 5;
"""
df_pe_re_nhat = pd.read_sql_query(query_cheap_pe, engine)
display(df_pe_re_nhat)

# bắt đầu lọc ra 5 cổ phiếu có P/E lớn nhất (năm mới nhất)
query_expensive_pe = """
WITH LatestData AS (
    SELECT 
        Ticker,
        year,
        "P/E",
        ROW_NUMBER() OVER(PARTITION BY Ticker ORDER BY year DESC) as rn 
    FROM 
        financial_ratio
    WHERE 
        "P/E" IS NOT NULL
)
SELECT 
    Ticker,
    year,
    "P/E"
FROM 
    LatestData
WHERE 
    rn = 1 
    AND "P/E" > 0  -- Chỉ lấy P/E đắt, không lấy P/E âm (lỗ)
ORDER BY
    "P/E" DESC
LIMIT 5;
"""
df_expensive_pe = pd.read_sql_query(query_expensive_pe, engine)
display(df_expensive_pe)

,Ticker,year,P/E
0,VRE,2024,9.75
1,HPG,2024,11.82
2,VNM,2024,13.02
3,VIC,2024,13.21
4,GAS,2024,14.44


,Ticker,year,P/E
0,MSN,2024,52.98
1,GVR,2024,30.00
2,FPT,2024,24.62
3,MWG,2024,23.62
4,PLX,2024,16.24
